this bot trades moving averages in the forex market using iqoption's API, it buys when the short term average is higher than the long term average and vice versea, this is only for learning purposes, not to be used on your real account, do so at your own risk




# DOWNLOADING AND IMPORTING LIBRARIES

In [ ]:
!pip install iqoptionapi
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
# import talib
!pip install -U git+git://github.com/Lu-Yi-Hsun/iqoptionapi.git
import time
from iqoptionapi.stable_api import IQ_Option
from talib import *

(Reading database ... 161006 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

  Cloning git://github.com/Lu-Yi-Hsun/iqoptionapi.git to /tmp/pip-req-build-kkca7z22
  Running command git clone -q git://github.com/Lu-Yi-Hsun/iqoptionapi.git /tmp/pip-req-build-kkca7z22
  Created wheel for iqoptionapi: filename=iqoptionapi-6.8.9.1-cp37-none-any.whl size=57158 sha256=f353f98e794d93bf0ef9cfa36bfd9fe0803a497cc1603975d44eb91c65516aec
  Stored in directory: /tmp/pip-ephem-wheel-cache-7aagfq6i/wheels/b7/05/a7

# LOGGING IN AND GETTING ACCOUNT DETAILS

In [ ]:
account2=IQ_Option("#############@gmail.com", "#########")
account2.connect()
print(account2.get_balance_id())
goal="AUDUSD"
print(IQ_Option.__version__)
account2.get_balance()

342579056
6.8.9.1


12.95

# CHECKING ALL OPEN ASSETS

In [ ]:
# ALL_Asset=account2.get_all_open_time()
# #check if open or not
# print(ALL_Asset["forex"]["EURUSD"]["open"]) 
# print(ALL_Asset["cfd"]["FACEBOOK"]["open"])#Stock,Commodities,ETFs
# print(ALL_Asset["crypto"]["BTCUSD-L"]["open"])
# print(ALL_Asset["digital"]["EURUSD-OTC"]["open"])

# #Binary have two diffenence type:"turbo","binary"
# print(ALL_Asset["turbo"]["EURUSD-OTC"]["open"])
# print(ALL_Asset["binary"]["EURUSD-OTC"]["open"])


# #!!!! exception ""
# print(ALL_Asset["binary"]["not exist asset"]["open"])#it will return "{}" a None of the dict

# #!!!!print all!!!!
# for type_name, data in ALL_Asset.items():
#     for Asset,value in data.items():
#         print(type_name,Asset,value["open"])

# SETTING MARKET


In [ ]:
instrument_type="forex"
instrument_id="EURUSD"
goal = instrument_id
size = 7200
maxdict = 400
side="sell"#input:"buy"/"sell"
amount=account2.get_balance() -1#input how many Amount you want to play
ID_Name="order_id"

#"leverage"="Multiplier"
leverage=50#you can get more information in get_available_leverages()

type="market"#input:"market"/"limit"/"stop"

#for type="limit"/"stop"

# only working by set type="limit"
limit_price=None#input:None/value(float/int)

# only working by set type="stop"
stop_price=None#input:None/value(float/int)

#"percent"=Profit Percentage
#"price"=Asset Price
#"diff"=Profit in Money
#input:None/value(float/int)

take_profit_kind=None#input:None/"price"/"diff"/"percent"
take_profit_value=None#input:None/value(float/int)

#"use_trail_stop"="Trailing Stop"
use_trail_stop=False#True/False

#"auto_margin_call"="Use Balance to Keep Position Open"
auto_margin_call=True#True/False
#if you want "take_profit_kind"&
#            "take_profit_value"&
#            "stop_lose_kind"&
#            "stop_lose_value" all being "Not Set","auto_margin_call" need to set:True

use_token_for_commission=False#True/False
#"position_id"/"order_id"
stop_lose_kind="percent"
stop_lose_value=20
take_profit_kind="percent"
take_profit_value=40
use_trail_stop=False
auto_margin_call=True


In [ ]:
account2.start_candles_stream(goal,size,maxdict)

# FOR NEW FRESH TRADE

In [ ]:
# I_want_money.start_candles_stream(goal,size,maxdict)
# print("Start EMA Sample")
import numpy as np
timeperiod=6
timeperiod1=14
buying = False
selling = False

# not = True
while True:
    candles=account2.get_realtime_candles(goal,size)
    
    inputs = {
        'open': np.array([]),
        'high': np.array([]),
        'low': np.array([]),
        'close': np.array([]),
        'volume': np.array([])
    }
    for timestamp in candles:
        
        inputs["open"]=np.append(inputs["open"],candles[timestamp]["open"] )
        inputs["high"]=np.append(inputs["open"],candles[timestamp]["max"] )
        inputs["low"]=np.append(inputs["open"],candles[timestamp]["min"] )
        inputs["close"]=np.append(inputs["open"],candles[timestamp]["close"] )
        inputs["volume"]=np.append(inputs["open"],candles[timestamp]["volume"] )
        
 
    print("Show EMA")
    sixSMA = SMA(inputs["close"], timeperiod=timeperiod)[-1]
    FORTEENSMA = SMA(inputs["close"], timeperiod=timeperiod1)[-1]
    closee = inputs["close"][-1]
    print("closing price ---->" ,closee ," sixperiod ---->" , sixSMA , " tenperiod ----> " , FORTEENSMA , 'order id---->',order_id )
    if sixSMA>tenSMA:
      if selling == True:
        print(account2.close_position(order_id))
      if buying == False:
        check,order_id=account2.buy_order(instrument_type=instrument_type, instrument_id=instrument_id,
            side='buy', amount=amount,leverage=leverage,
            type=type,limit_price=limit_price, stop_price=stop_price,
            stop_lose_value=stop_lose_value, stop_lose_kind=stop_lose_kind,
            take_profit_value=take_profit_value, take_profit_kind=take_profit_kind,
            use_trail_stop=use_trail_stop, auto_margin_call=auto_margin_call,
            use_token_for_commission=use_token_for_commission)
        
      
      
      buying = True
      selling = False
    elif sixSMA<tenSMA:
      if buying == True:
        print(account2.close_position(order_id))
      if selling == False:
        check,order_id=account2.buy_order(instrument_type=instrument_type, instrument_id=instrument_id,
            side='sell', amount=amount,leverage=leverage,
            type=type,limit_price=limit_price, stop_price=stop_price,
            stop_lose_value=stop_lose_value, stop_lose_kind=stop_lose_kind,
            take_profit_value=take_profit_value, take_profit_kind=take_profit_kind,
            use_trail_stop=use_trail_stop, auto_margin_call=auto_margin_call,
            use_token_for_commission=use_token_for_commission)
        
        buying = False
        selling = True
        
      
 
      print("\n")
      time.sleep(1)
 
account2.stop_candles_stream(goal,size)

Show EMA
closing price ----> 1.20871  sixperiod ----> 1.1857866666666665  tenperiod ---->  1.1780564285714266 order id----> 2499129542


Show EMA
closing price ----> 1.20871  sixperiod ----> 1.1857866666666665  tenperiod ---->  1.1780564285714266 order id----> 2499136254


Show EMA
closing price ----> 1.20871  sixperiod ----> 1.1857866666666665  tenperiod ---->  1.1780564285714266 order id----> 2499136254


Show EMA
closing price ----> 1.20871  sixperiod ----> 1.1857866666666665  tenperiod ---->  1.1780564285714266 order id----> 2499136254


Show EMA
closing price ----> 1.20871  sixperiod ----> 1.1857866666666665  tenperiod ---->  1.1780564285714266 order id----> 2499136254


Show EMA
closing price ----> 1.20871  sixperiod ----> 1.1857866666666665  tenperiod ---->  1.1780564285714266 order id----> 2499136254


Show EMA
closing price ----> 1.20871  sixperiod ----> 1.1857866666666665  tenperiod ---->  1.1780564285714266 order id----> 2499136254


Show EMA
closing price ----> 1.20871  six

# FOR A PREVIOUSLY BUY

In [ ]:
# import numpy as np
# timeperiod=6
# timeperiod1=14
# buying = True
# selling = False
# order_id = 2498436941 # previous order id
# # not = True
# while True:
#     candles=account2.get_realtime_candles(goal,size)
    
#     inputs = {
#         'open': np.array([]),
#         'high': np.array([]),
#         'low': np.array([]),
#         'close': np.array([]),
#         'volume': np.array([])
#     }
#     for timestamp in candles:
        
#         inputs["open"]=np.append(inputs["open"],candles[timestamp]["open"] )
#         inputs["high"]=np.append(inputs["open"],candles[timestamp]["max"] )
#         inputs["low"]=np.append(inputs["open"],candles[timestamp]["min"] )
#         inputs["close"]=np.append(inputs["open"],candles[timestamp]["close"] )
#         inputs["volume"]=np.append(inputs["open"],candles[timestamp]["volume"] )
        
 
#     print("Show SMA")
#     sixSMA = SMA(inputs["close"], timeperiod=timeperiod)[-1]
#     tenSMA = SMA(inputs["close"], timeperiod=timeperiod1)[-1]
#     closee = inputs["close"][-1]
#     print("closing price ---->" ,closee ," sixperiod ---->" , sixSMA , " tenperiod ----> " , tenSMA , 'order id---->',order_id)
#     if sixSMA>tenSMA:
#       if selling == True:
#         print(account2.close_position(order_id))
#       if buying == False:
#         check,order_id=account2.buy_order(instrument_type=instrument_type, instrument_id=instrument_id,
#             side='buy', amount=amount,leverage=leverage,
#             type=type,limit_price=limit_price, stop_price=stop_price,
#             stop_lose_value=stop_lose_value, stop_lose_kind=stop_lose_kind,
#             take_profit_value=take_profit_value, take_profit_kind=take_profit_kind,
#             use_trail_stop=use_trail_stop, auto_margin_call=auto_margin_call,
#             use_token_for_commission=use_token_for_commission)
        
      
      
#       buying = True
#       selling = False
#     elif sixSMA<tenSMA:
#       if buying == True:
#         print(account2.close_position(order_id))
#       if selling == False:
#         check,order_id=account2.buy_order(instrument_type=instrument_type, instrument_id=instrument_id,
#             side='sell', amount=amount,leverage=leverage,
#             type=type,limit_price=limit_price, stop_price=stop_price,
#             stop_lose_value=stop_lose_value, stop_lose_kind=stop_lose_kind,
#             take_profit_value=take_profit_value, take_profit_kind=take_profit_kind,
#             use_trail_stop=use_trail_stop, auto_margin_call=auto_margin_call,
#             use_token_for_commission=use_token_for_commission)
        
#         buying = False
#         selling = True
        
      
 
#       print("\n")
#       time.sleep(1)
 
# account2.stop_candles_stream(goal,size)

# FOR A PREVIOUSLY SELL

In [ ]:
# import numpy as np
# timeperiod=6
# timeperiod1=14
# buying = False
# selling = True
# order_id = 2498436941 # previous order id
# # not = True
# while True:
#     candles=account2.get_realtime_candles(goal,size)
    
#     inputs = {
#         'open': np.array([]),
#         'high': np.array([]),
#         'low': np.array([]),
#         'close': np.array([]),
#         'volume': np.array([])
#     }
#     for timestamp in candles:
        
#         inputs["open"]=np.append(inputs["open"],candles[timestamp]["open"] )
#         inputs["high"]=np.append(inputs["open"],candles[timestamp]["max"] )
#         inputs["low"]=np.append(inputs["open"],candles[timestamp]["min"] )
#         inputs["close"]=np.append(inputs["open"],candles[timestamp]["close"] )
#         inputs["volume"]=np.append(inputs["open"],candles[timestamp]["volume"] )
        
 
#     print("Show SMA")
#     sixSMA = SMA(inputs["close"], timeperiod=timeperiod)[-1]
#     tenSMA = SMA(inputs["close"], timeperiod=timeperiod1)[-1]
#     closee = inputs["close"][-1]
#     print("closing price ---->" ,closee ," sixperiod ---->" , sixSMA , " tenperiod ----> " , tenSMA , 'order id---->',order_id)
#     if sixSMA>tenSMA:
#       if selling == True:
#         print(account2.close_position(order_id))
#       if buying == False:
#         check,order_id=account2.buy_order(instrument_type=instrument_type, instrument_id=instrument_id,
#             side='buy', amount=amount,leverage=leverage,
#             type=type,limit_price=limit_price, stop_price=stop_price,
#             stop_lose_value=stop_lose_value, stop_lose_kind=stop_lose_kind,
#             take_profit_value=take_profit_value, take_profit_kind=take_profit_kind,
#             use_trail_stop=use_trail_stop, auto_margin_call=auto_margin_call,
#             use_token_for_commission=use_token_for_commission)
        
      
      
#       buying = True
#       selling = False
#     elif sixSMA<tenSMA:
#       if buying == True:
#         print(account2.close_position(order_id))
#       if selling == False:
#         check,order_id=account2.buy_order(instrument_type=instrument_type, instrument_id=instrument_id,
#             side='sell', amount=amount,leverage=leverage,
#             type=type,limit_price=limit_price, stop_price=stop_price,
#             stop_lose_value=stop_lose_value, stop_lose_kind=stop_lose_kind,
#             take_profit_value=take_profit_value, take_profit_kind=take_profit_kind,
#             use_trail_stop=use_trail_stop, auto_margin_call=auto_margin_call,
#             use_token_for_commission=use_token_for_commission)
        
#         buying = False
#         selling = True
        
      
 
#       print("\n")
#       time.sleep(1)
 
# account2.stop_candles_stream(goal,size)